Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploying a ML module on IoT Edge Device


In this notebook, we introduce the steps of deploying an ML module on [Azure IoT Edge](https://docs.microsoft.com/en-us/azure/iot-edge/how-iot-edge-works). The purpose is to deploy a trained machine learning model to the edge device. When the input data is generated from a particular process pipeline and fed into the edge device, the deployed model is able to make predictions right on the edge device without accessing to the cloud. 


## Outline<a id="BackToTop"></a>
- [Prerequisites](#prerequisite)
- [Step 1: Create an IoT Hub and Register an IoT Edge device](#step1)
- [Step 2: Provision and Configure IoT Edge Device](#step2)
- [Step 3: Deploy ML Module on IoT Edge Device](#step3)
- [Step 4: Test ML Module](#step4)
- [Clean up resource](#cleanup)

## Prerequisites <a id="Prerequisite"></a>

- Satisfy the requirment specified in Sections `Prerequisites` and `Setup` in the repo's [README page](./README.md).

- Build the trained ML Model into a docker image. 
    
    - You have two options to satisfy this requirment. The first option is to complete all the notebooks from [00_AML_Configuration.ipynb](./00_AMLConfiguration.ipynb) through [04_CreateImage.ipynb](./04_CreateImage.ipynb). In this tutorial, we show the steps when using this option. The second option is to use a prebuilt docker image created by the user. Instructions need to be adjusted to proceed with this option.

In [ ]:
import sys
import pandas as pd
import requests
import numpy as np
import json
import docker
import time

from azureml.core import Workspace
from azureml.core.image import Image
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
from dotenv import set_key, get_key, find_dotenv

from utilities import text_to_json, get_auth

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

## Step 1 Create an IoT Hub and Register an IoT Edge device  <a id="step1"></a>

For more infromation, please check Sections `Create an IoT hub` and `Register an IoT Edge device` in document [Deploy Azure IoT Edge on a simulated device in Linux or MacOS - preview](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-simulate-device-linux). When creating IoT hub, we assume you use the same resource group as the one created in [00_AML_Configuration.ipynb](./00_AML_Configuration.ipynb). 

### Get workspace

Load existing workspace from the config file.

In [ ]:
ws = Workspace.from_config(auth=get_auth(env_path))
print(ws.name, ws.resource_group, ws.location, sep="\n")

In [83]:
iot_hub_name = "<YOUR_IOT_HUB_NAME>" # a UNIQUE name is required, e.g. "fstlstnameiothub". Avoid too simple name such as "myiothub".
device_id = "<YOUR_EDGE_DEVICE_NAME>" # the name you give to the edge device. e.g. device_id = 'mydevice'
module_name = "<YOUR_MODULE_NAME>"   # the module name. e.g. module_name = 'mymodule'

In [ ]:
set_key(env_path, "iot_hub_name", iot_hub_name)
set_key(env_path, "device_id", device_id)
set_key(env_path, "module_name", module_name)

In [ ]:
iot_hub_name = get_key(env_path, 'iot_hub_name')
device_id = get_key(env_path, 'device_id')
module_name = get_key(env_path, 'module_name')
resource_group = get_key(env_path, 'resource_group')
image_name = get_key(env_path, 'image_name')

### Install az-cli iot extension 

In [ ]:
accounts = !az account list --all -o tsv
if "Please run \"az login\" to access your accounts." in accounts[0]:
    !az login -o table
else:
    print("Already logged in")

### Create IoT Hub

The following code creates a free F1 hub with name `iot_hub_name` in the resource group `resource_group`.

Common issue and resolution are shown below.

(1) See error message "Resource group `<resource_group>` could not be found". You may forget to set which Azure subscription to use. Run command `az account set --subscription <subscription_id>` to set your subscription.

(2) See error message "Max number of Iot Hubs exceeded for sku = Free, Max Allowed = 1, Current = 2 in the subscription Id: `<subscription_id>`." This error message indicates that the quota is reached for creating the specific type of IoT Hub resource in Azure.

In [ ]:
!az extension add --name azure-cli-iot-ext
!az iot hub list --resource-group $resource_group -o table

In [ ]:
!az iot hub create --resource-group $resource_group --name $iot_hub_name --sku F1 

### Register an IoT Edge device

In the Azure cloud shell, enter the following command to create a device with name `device_id` in your iot hub.

In [ ]:
!az iot hub device-identity create --hub-name $iot_hub_name --device-id $device_id --edge-enabled -g $resource_group

Obtain `device_connection_string`. It will be used in the next step.

In [ ]:
json_data = !az iot hub device-identity show-connection-string --device-id $device_id --hub-name $iot_hub_name -g $resource_group
print(json_data)

In [ ]:
device_connection_string = json.loads(''.join([i for i in json_data if 'WARNING' not in i]))['connectionString']
print(device_connection_string)

## Step 2 Provision and Configure IoT Edge Device  <a id="step2"></a>

In this tutorial, we use a NC6 Ubuntu Linux VM as the edge device, which is the same Linux VM where you run the current notebook. The goal is to configure the edge device so that it can run [Docker](https://docs.docker.com/v17.12/install/linux/docker-ee/ubuntu), [nvidia-docker](https://github.com/NVIDIA/nvidia-docker), and [IoT Edge runtime](https://docs.microsoft.com/en-us/azure/iot-edge/how-to-install-iot-edge-linux). If another device is used as the edge device, instructions need to be adjusted accordingly. 

### Register Microsoft key and software repository feed

Prepare your device for the IoT Edge runtime installation.

In [91]:
# Install the repository configuration. Replace <release> with 16.04 or 18.04 as appropriate for your release of Ubuntu.
release = !lsb_release -r
release  = release[0].split('\t')[1]
print(release)

16.04


In [ ]:
!curl https://packages.microsoft.com/config/ubuntu/$release/prod.list > ./microsoft-prod.list

In [ ]:
# Copy the generated list.
!sudo cp ./microsoft-prod.list /etc/apt/sources.list.d/

In [ ]:
#Install Microsoft GPG public key
!curl https://packages.microsoft.com/keys/microsoft.asc | gpg --dearmor > microsoft.gpg
!sudo cp ./microsoft.gpg /etc/apt/trusted.gpg.d/

### Install the Azure IoT Edge Security Daemon

The IoT Edge security daemon provides and maintains security standards on the IoT Edge device. The daemon starts on every boot and bootstraps the device by starting the rest of the IoT Edge runtime.
The installation command also installs the standard version of the iothsmlib if not already present.

In [ ]:
# Perform apt update.
!sudo apt-get update

In [ ]:
# Install the security daemon. The package is installed at /etc/iotedge/.
!sudo apt-get install iotedge -y --no-install-recommends

### Configure the Azure IoT Edge Security 

Configure the IoT Edge runtime to link your physical device with a device identity that exists in an Azure IoT hub.
The daemon can be configured using the configuration file at /etc/iotedge/config.yaml. The file is write-protected by default, you might need elevated permissions to edit it.

In [ ]:
# Manual provisioning IoT edge device
!sudo sed -i "s#\(device_connection_string: \).*#\1\"$device_connection_string\"#g" /etc/iotedge/config.yaml

In [ ]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(10) # Wait 10 seconds for iotedge to restart

In [ ]:
# restart the daemon again
!sudo systemctl restart iotedge

### Disable process identification

While in preview, Azure Machine Learning does not support the process identification security feature enabled by default with IoT Edge. Below are the steps to disable it. This is however not suitable for use in production. These steps are only necessary on Linux, as you will have completed this during the Windows Edge runtime installation.

To disable process identification on your IoT edge device, you'll need to provide the ip address and port for workload_uri and management_uri in the connect section of the IoT Edge daemon configuration.

Get the IP address first. Enter `ifconfig` in your command line and copy the IP address of the docker0 interface.

For more information, please check section `Disable process identification` of document [Tutorial: Deploy Azure Machine Learning as an IoT Edge module ](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning).

In [1]:
output_data = !ifconfig
ip_docker0 = output_data[1].split(":")[1].split()[0]
print(ip_docker0)

172.17.0.1


In [ ]:
management_uri = 'http://{}:15580'.format(ip_docker0)
workload_uri = 'http://{}:15581'.format(ip_docker0)

In [ ]:
!sudo sed -i "s#\(management_uri: \).*#\1\"$management_uri\"#g" /etc/iotedge/config.yaml
!sudo sed -i "s#\(workload_uri: \).*#\1\"$workload_uri\"#g" /etc/iotedge/config.yaml

In [ ]:
# restart the daemon
!sudo systemctl restart iotedge
time.sleep(10) # Wait 10 seconds for iotedge to restsrt

### Verify successful installation

In [ ]:
# check the status of the IoT Edge Daemon
!systemctl status iotedge

In [ ]:
# Examine daemon logs
!journalctl -u iotedge --no-pager --no-full

When you run `docker ps` command in the edge device, you should see `edgeAgent` container is up running.

In [ ]:
!docker ps

### (Optional) Alternative Approach to Configure IoT Edge Device

Use this approach if your edge device is a different server than the host server. Note that your edge device must satisfy following prequequisites:

- Linux (x64)
- Docker installed

Step 1: run appropriate cells above to get the value for following variable.

- device_connection_string


Step 2: run approprate commands on the edge device to get values for following variables.

- release
- management_uri
- workload_uri

Step 3: run next cell to generate *deviceconfig.sh* file. 

Step 4: run all the commands in *deviceconfig.sh* file on your edge device. 


In [ ]:
file = open('./deviceconfig_template.sh')
contents = file.read()
contents = contents.replace('__release', release)
contents = contents.replace('__device_connection_string', device_connection_string)
contents = contents.replace('__management_uri', management_uri)
contents = contents.replace('__workload_uri', workload_uri)

with open('./deviceconfig.sh', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

## Step 3: Deploy the ML module  <a id="step3"></a>

For more information, please check instructions from [this doc](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where).

In [ ]:
docker_client = docker.APIClient(base_url='unix://var/run/docker.sock')

In [ ]:
# get the image from workspace in case the 'image' object is not in the memory
image_name = get_key(env_path, 'image_name')
image = ws.images[image_name]

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(reg_name))
print('Username: {}'.format(username))
print('Password: {}'.format(password))

In [ ]:
file = open('MLmodule_deployment_template.json')
contents = file.read()
contents = contents.replace('__MODULE_NAME', module_name)
contents = contents.replace('__REGISTRY_NAME', reg_name)
contents = contents.replace('__REGISTRY_USER_NAME', username)
contents = contents.replace('__REGISTRY_PASSWORD', password)
contents = contents.replace('__REGISTRY_IMAGE_LOCATION', image.image_location)
with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $device_id \
                         --hub-name $iot_hub_name \
                         --content deployment.json \
                         -g $resource_group

You can check the logs of the ML module with using the command in the next cell. **Note that if your edge device differs from the host server, you need to run this command on the edge device.** 

Common issue and resolution are shown below.

An error message "Error: No such container: module_name" is shown. Resolution - Please wait for a couple minutes and run this command again. The container is starting up. 

In [ ]:
!docker logs -f $module_name

In [ ]:
def get_id(container_name):
    contents = docker_client.containers()
    for cont in contents:
        if container_name in cont['Names'][0]:
            return cont["Id"]
    return None

In [ ]:
d_id = get_id(module_name)
while d_id is None:
    d_id = get_id(module_name)
    time.sleep(1)

In [ ]:
logs = docker_client.attach(d_id, stream=True, logs=True)

In [ ]:
# keep running this cell until the log contains "Using TensorFlow backend", which indicates the container is up running.
for l in logs:
    msg = l.decode('utf-8')
    print(msg)
    if "Opened module client connection" in msg:
        break   

When you run `docker ps` command in the edge device, you should see there are three containers running: `edgeAgent`, `edgeHub`, and the container with name `module_name`.

## Step 4: Test ML Module <a id="step4"></a>
We now test the ML Module from iot Edge device.

In [55]:
dupes_test_path = './data_folder/dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

"get total number of items on json object?.  possible duplicate: length of javascript object (ie. associative array)   i have an object similar to this one:  i'm trying to get it's length, the problem is that jsonarray.length returns 5 instead of 3 (which is the total items it has). the array is relatively long (has 1000x2000 items) and this must be done a lot of times every second. how can i get the number of items more efficiently?"

In [56]:
jsontext = text_to_json(text_to_score)

Let's try a few more duplicate questions and display their top 3 original matches. Let's first get the scoring URL and and API key for the web service.

In [80]:
scoring_url = 'http://localhost:5001/score'

In [ ]:
# call the web service end point
headers = {'Content-Type':'application/json'}
response = requests.post(scoring_url, data=jsontext, headers=headers)
response

In [68]:
prediction = json.loads(response.content.decode('ascii'))
prediction

'[[11922383, 11922384, 0.9397743269422142], [5223, 6700, 0.9192726368860034], [126100, 4889658, 0.8424747207316404], [23667086, 23667087, 0.6399820273867503], [6491463, 6491621, 0.21430600871807984], [12953704, 12953750, 0.17224920880773398], [1129216, 1129270, 0.1615964737435373], [19590865, 19590901, 0.15327286262230572], [171251, 171256, 0.15267175719683257], [695050, 695053, 0.1327395820659991], [3127429, 3127440, 0.12390130949759141], [2901102, 2901298, 0.10336326500861436], [4968406, 4968448, 0.10249638505076407], [4057440, 4060176, 0.07941774037669798], [728360, 728694, 0.07692824139869399], [7364150, 7364307, 0.05547557395333095], [4616202, 4616273, 0.04798803949633269], [1451009, 1451043, 0.04765288198385024], [1885557, 1885660, 0.04088269195896825], [111102, 111111, 0.03489096061980277], [979256, 979289, 0.022370639909675034], [85992, 86014, 0.02029824630664947], [2274242, 2274327, 0.01701921667517765], [22519784, 22519785, 0.014798770726383218], [1069666, 1069840, 0.01362085

In [ ]:
dupes_to_score = dupes_test.iloc[:5,4]

In [ ]:
results = [
    requests.post(scoring_url, data=text_to_json(text), headers=headers)
    for text in dupes_to_score
]

Let's print top 3 matches for each duplicate question.

In [ ]:
[eval(results[i].json())[0:3] for i in range(0, len(results))]

Next let's quickly check what the request response performance is for the deployed model on IoT edge device.

In [ ]:
text_data = list(map(text_to_json, dupes_to_score))  # Retrieve the text data

In [ ]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, data=text, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))